In [1]:
import ray
import torch
from rllib_setup import get_env_continuous

In [2]:
ray.init()

2023-07-18 18:04:13,994	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


In [3]:
env = get_env_continuous()

In [4]:
env_name = "VJS"

In [5]:
ray.rllib.utils.check_env(env)

In [6]:
from ray.tune import register_env
register_env(env_name,lambda config: get_env_continuous())

In [8]:
from ray.rllib.policy.policy import PolicySpec
test_env = get_env_continuous()
obs_space = test_env.observation_space
act_space = test_env.action_space
def policies(agent_ids):
    return {
        str(i): PolicySpec(
            observation_space=obs_space,
            action_space=act_space,
            config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }

In [9]:
from ray.rllib.algorithms.maddpg.maddpg import MADDPGConfig
config = MADDPGConfig()
print(config.replay_buffer_config)  
replay_config = config.replay_buffer_config.update(  
    {
        "capacity": 100000,
        "prioritized_replay_alpha": 0.8,
        "prioritized_replay_beta": 0.45,
        "prioritized_replay_eps": 2e-6,
    }
)
config.training(replay_buffer_config=replay_config)   
config = config.resources(num_gpus=0)   
config = config.rollouts(num_rollout_workers=4)   
config = config.environment(env_name)
config = config.multi_agent(policies=policies(test_env._agent_ids),policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: str(agent_id))
algo = config.build()  
algo.train()

{'type': 'MultiAgentReplayBuffer', 'prioritized_replay': -1, 'capacity': 1000000, 'replay_mode': 'lockstep'}


(RolloutWorker pid=1858872) /home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/algorithms/maddpg/maddpg_tf_policy.py:357: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
(RolloutWorker pid=1858872)   out = tf1.layers.dense(out, units=hidden, activation=activation)
(RolloutWorker pid=1858872) /home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/algorithms/maddpg/maddpg_tf_policy.py:359: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
(RolloutWorker pid=1858872)   out = tf1.layers.dense(feature, units=1, activation=None)
(RolloutWorker pid=1858048) 2023-07-18 18:06:37,907	ERROR worker.py:861 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=1858048, ip=192.168.3.6, actor_id=b29ab9965315be97787dfc7901000000, repr=<ray.rllib.ev

AssertionError: 